# A solution for NSUCRYPTO 2020
- Problem 3: _Hidden RSA_
- By: _ndh_

Let $f$ denote `Encr`.

Since we have an oracle that returns $y = f(x) = x^e \text{ mod } n$ for arbitrary $x$ of our choice, we are able to compute as many multiples of $n$ as we want by using the fact that $f(x_1)f(x_2) = f(x_1x_2) \text { mod } n$. For example, knowing that $f(2)^2 = f(4) \text{ mod } n$, we can make queries for $f(2)$ and $f(4)$ and compute $f(2)^2 - f(4)$ which should be a multiple of $n$. $n$ can then be recovered by applying the standard Euclidean Greatest Common Divisor algorithm.

Here are $f(2)$, $f(4)$ and $f(8)$ returned from the server:

In [1]:
y2 = 50154912289039335014669339773308393642658123228965873078737860474494117389068
y4 = 34176590322694690833975364940063423615063159848675783675025873390206977645476
y8 = 52560386591865237573584602609517662189426443148418873287042585337399092751771

Let's find $n$:

In [2]:
from Crypto.Util.number import GCD  # https://pypi.org/project/pycryptodome/
k1n = y2*y2 - y4
k2n = y2*y4 - y8
n = GCD(k1n, k2n)  # this is in fact GCD(k1, k2) * n

# brute-force GCD(k1, k2).
for i in range(2**(n.bit_length() - y2.bit_length() + 1), 1, -1):
    if n % i == 0:
        n = n // i
        break
        
print(n.bit_length())

256


As we can see, $n$ is only 256 bit long. Factoring a 256-bit integer is not a hard task nowadays. For example, the general `factor` function of [SageMath](https://www.sagemath.org/) (there are probably better tools around) can do the job in a few minutes on a common laptop.

In [3]:
%%time
from sage.all import factor
(p, _), (q, _) = factor(n)

CPU times: user 5min 29s, sys: 527 ms, total: 5min 29s
Wall time: 5min 29s


Now, we compute the secret exponent $d$ by solving 2 discrete logarithm problems over the 2 prime fields $\mathbb{F}_p$ and $\mathbb{F}_q$. Again, since $p$ and $q$ are only 128-bit long, the problems are not hard.

In [4]:
%%time
from sage.all import GF
Fp = GF(p)
Fq = GF(q)
dp = Fp(2).log(Fp(y2))  # y2^dp = 2 mod p
dq = Fq(2).log(Fq(y2))  # y2^dq = 2 mod q

CPU times: user 3min 16s, sys: 405 ms, total: 3min 17s
Wall time: 3min 16s


Now, let's decrypt the given ciphertext $y$ (by solving for $x \text{ mod } p$ and $x \text { mod } q$ first, then using the Chinese Remainder Theorem to derive the plaintext $x \text{ mod } n$):

In [5]:
from sage.all import crt
y = 71511896681324833458361392885184344933333159830863878600189212073777582178173
xp = Fp(y) ** dp
xq = Fq(y) ** dq
x = crt(int(xp), int(xq), p, q)
print(x)

202010181600


The plaintext is the starting time of the second round in UTC+7: 16h00 18/10/2020.